In [32]:
import deeptrack as dt
from deeptrack.models.yolo import utils
import numpy as np
import matplotlib.pyplot as plt

from simulation import *

In [33]:
sim= MultiParticle(
    
    
    n_particles = lambda: int(np.random.rand()*10) + 15,
        
    
)
starting_value = dt.Value(lambda: np.ones((IMAGE_SIZE, IMAGE_SIZE)))

sim= starting_value >> MultiParticle(
    n_particles = lambda: int(np.random.rand()*10) + 15,
)

particle =  sim[0]
pos =  sim[1]
rad =  sim[2]






In [203]:
print(pos.resolve())
print(rad.resolve())

Image(array([[95.4388479 , 92.83221213],
       [17.24904284, 27.0755127 ],
       [37.08994345, 48.68179895],
       [21.74113487, 62.42353708],
       [41.86144144,  8.30276784],
       [73.06197007, 70.77536092],
       [ 4.53360205, 30.71499169],
       [68.05067905, 78.98171148],
       [60.11809015, 59.88783825],
       [78.15844424, 66.01043536],
       [63.57074131, 72.04571455],
       [32.9966145 , 81.97902834],
       [53.93480455, 34.87411615],
       [19.97358733,  4.48004108],
       [97.44132705, 43.51237193],
       [45.77577548, 53.55694118],
       [ 8.40493573, 27.08631213],
       [31.13276837, 63.81474919],
       [93.37053933, 63.06279786]]))
Image(array([[2.57831734],
       [2.06932651],
       [2.5275487 ],
       [2.14278909],
       [2.34966374],
       [2.55172   ],
       [2.14496138],
       [2.83655537],
       [2.52517086],
       [2.29838099],
       [2.6331675 ],
       [2.55334598],
       [2.3646416 ],
       [2.24667131],
       [2.73603216],
      

In [38]:
gradient = dt.IlluminationGradient(
    gradient=[.5e-3, 0e-3],#lambda: 1e-3 + np.random.randn(2) * 1e-3,
)

class Normalize(dt.Feature):
    def __init__(
        self, **kwargs
    ):
        super().__init__( **kwargs)

    def get(self, image, **kwargs):
        image = image / np.max(image)
        return image

normalization = Normalize()

gauss = dt.Gaussian(mu = 0, sigma = .01)

import scipy
#Smoothing
kernel = np.ones((3, 3)) / 15
smoothing = dt.Lambda(lambda: lambda image: scipy.ndimage.filters.convolve(image, kernel)) 

particle = particle >>  smoothing  >>gauss >> normalization 

In [273]:
"""def get_label(a, b):
    a = a.resolve()
    b = b.resolve()
    n = int(np.shape(a)[0])
    label = np.zeros((n,4))

    for i in range(n):
        label[i,:] = [int(a[i, 0]), int(a[i, 1]), b[i], b[i]]

    
    return label #(np.asarray(a.resolve()), np.asarray(b.resolve()))"""

def get_label(rad, pos):
    rad = np.array(rad)
    pos = np.array(pos)
        
    n = np.shape(rad)[0]  
    label = np.zeros((n, 4))

    for i in range ( n ):
        label[i, : ] = [int(pos[i, 0]), int(pos[i, 1]), int(rad[i,0]), int(rad[i,0])]
    
    #print(label)
    return label

In [279]:
pos_and_rad = pos & rad

#bboxes = pos_and_rad >> get_label
label_func = dt.Lambda(lambda : lambda pos, rad: get_label(rad, pos), rad=rad, pos=pos)
#bboxes.resolve()
#data = particle & bboxes
#print(data.resolve())
a = label_func

print(a.resolve())

[]


In [230]:
NUMBER_OF_IMAGES = 5

for _ in range(NUMBER_OF_IMAGES):
    particle.update()
    pos = pos.update()
    rad = rad.update()
    pos_and_rad = pos & rad 

    bboxes = label_func.resolve

    label = bboxes
    print(np.array(label))
    image_of_particle = particle.resolve()
    

    
    n = np.shape(label)[0]
    position_of_particles = label[:,0:2]
    print(position_of_particles)
    radius_of_particles = label[:,2] 
    
    plt.imshow(image_of_particle, cmap="gray", vmin = 0)
    plt.colorbar()
    ax = plt.gca()
    for i in range(int(n)):
        rect = plt.Rectangle((label[i,0]-label[i,2],label[i,1]-label[i,2]),2*label[i,2],2*label[i,2],linewidth=1,edgecolor='r',facecolor='none')
        # Add the patch to the Axes
        ax.add_patch(rect)
    
    
    #plt.scatter(position_of_particles[:,0], position_of_particles[:,1], marker='+', edgecolors="r", linewidth=2)
    plt.show()

NotImplementedError: Indexing with non-integer types not yet implemented

In [ ]:
from deeptrack.models.yolo.yolo import YOLOv3

model = YOLOv3(
    (256, 256, 1),
    2,
)

model.compile(optimizer="adam")

In [ ]:
from deeptrack.models.yolo.dataset import YoloDataGenerator

generator = YoloDataGenerator(
    feature=data,
    input_size=np.array([256]),
    num_class=2,
    label_function=lambda d: np.array(d[1:]),
    batch_size=4,
    min_data_size=100,
    max_data_size=200,
)

generator.anchors = model.anchors